<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/feature_extraction_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
! pip install -q langchain_community langchain_openai faiss-cpu langchain langchain_core langgraph pinecone PyPDF2 doi2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.

In [ ]:
"""
Paper Crawler

This script downloads research papers based on DOIs and saves them to a specified directory.
It starts with a seed paper and follows references to download more papers.
"""

import os
import time
import uuid
import json
import requests
import pandas as pd
from doi2pdf import doi2pdf
import PyPDF2
from langchain_community.document_loaders import WebBaseLoader
from typing import Dict, List, Any, Tuple, Optional, Union
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import pickle
from langchain_core.pydantic_v1 import Field, conlist, BaseModel

# Set environment variables for LangChain and OpenAI
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "irrigation-crawler"
os.environ["OPENAI_API_KEY"] = ""
# mg's acccount

In [ ]:
# save point for intermediate papers
DOWNLOAD_DIR = "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/Papers"


# save point for each csv_file per seed paper
OUTPUT_DIR = "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# LLM Configs

In [ ]:
class ReferenceItem(BaseModel):
    """A reference from a paper."""
    title: str = Field(description="Title of the referenced paper")
    authors: str = Field(description="Authors of the referenced paper")
    year: str = Field(description="Publication year")
    doi: str = Field(description="DOI if available, otherwise 'NA'")
    doi_url: str = Field(description="DOI URL if available, otherwise 'NA'")
    venue: str = Field(description="Publication venue (journal or conference)")

class ReferenceLLM(BaseModel):
    """List of references extracted from a paper."""
    reference_set: conlist(ReferenceItem, min_items=0) = Field(
        description="List of all references extracted from the paper"
    )

class RelevanceItem(BaseModel):
    relevance_score: int = Field(default=0, description="Score from 1-10 on relevance to irrigation scheduling")
    score_justification: str = Field(default="NA")

class RelevanceLLM(BaseModel):
    """Model for LLM feature extraction response"""
    feature_set: List[RelevanceItem] = Field(description="Scoring on relevance to irrigation scheduling")

class FeatureItem(BaseModel):
    """Model for storing extracted feature data"""
    irrigation_scheduling_method: str = Field(default="NA")
    sensor_type: str = Field(default="NA")
    sensor_threshold: str = Field(default="NA")
    number_of_sensors: str = Field(default="NA")
    evapotranspiration_level: str = Field(default="NA")
    daily_drainage: str = Field(default="NA")
    amount_irrigated: str = Field(default="NA")
    number_of_irrigation_events: str = Field(default="NA")
    irrigation_time: str = Field(default="NA")
    location: str = Field(default="NA")
    latitude: str = Field(default="NA")
    longitude: str = Field(default="NA")
    altitude: str = Field(default="NA")
    topography: str = Field(default="NA")
    farm_size: str = Field(default="NA")
    experiment_date: str = Field(default="NA")
    plant_type: str = Field(default="NA")
    number_of_plants: str = Field(default="NA")
    avg_fruit_per_plant: str = Field(default="NA")
    avg_fruit_weight: str = Field(default="NA")
    fruit_quality: str = Field(default="NA")
    vitamin_level: str = Field(default="NA")
    plant_yield: str = Field(default="NA")
    plant_height: str = Field(default="NA")
    water_productivity: str = Field(default="NA")

class FeatureLLM(BaseModel):
    """Model for LLM feature extraction response"""
    feature_set: List[FeatureItem] = Field(description="List of extracted irrigation features for each irrigation method")

FEATURE_EXTRACTION_INSTRUCTIONS = """
You are an assistant extracting irrigation experiment data from a research paper.

Extract ALL available information from the paper.

Identify all the variables experimented with in this paper. The variable(s) will often be one or more of the below features.
Each time any of these variables or a unique combination of their values are changed, it is considered one experiment.

If a paper reports multiple values for any variable, you must return a **separate object for each unique combination** instead of joining them with commas.

For each experiment, extract:

1. irrigation_scheduling_method: The specific method used for irrigation management. If it's deficit irrigation, make sure to return what kind of deficit irrigation.
2. sensor_type: Types of sensors used for irrigation management
3. sensor_threshold: Thresholds used for triggering irrigation
4. number_of_sensors: How many sensors were used
5. evapotranspiration_level: ET levels mentioned in the study
6. daily_drainage: Information about daily drainage measurements
7. amount_irrigated: Quantitative data on irrigation amounts
8. number_of_irrigation_events: How many irrigation events occurred
9. irrigation_time: When irrigation was applied
10. location: Study location
11. latitude: latitude of the study location
12. longitude: longitude of the study location
13. altitude: altitude or height above mean sea level
14. topography: the type of terrain of the farm used in the study (e.g. hilly, sloping, flat)
15. farm_size: size of the farm
16. experiment_date: date/time that this measurement/experiment was done.
17. plant_type: Crop or plant species studied
18. number_of_plants: Plant population or sample size
19. avg_fruit_per_plant: Average fruit count per plant
20. avg_fruit_weight: Average weight of fruits
21. fruit_quality: Information about fruit quality metrics
22. vitamin_level: Vitamin content data
23. plant_yield: Yield information
24. plant_height: Plant height data
25. water_productivity: Water use efficiency or productivity metrics

RULES:
- Return as an list of objects for the results of each experiment
- Each object must have exactly one value per field.
- If a variable (e.g., evapotranspiration_level) contains multiple values in the paper, you **must** create a separate object for each value.
  Do not combine multiple values into a single string.
  Always output all reported values as separate rows, even if it duplicates other columns.
- Extract ALL information even if there is no irrigation scheduling method
- For papers without irrigation methods, use irrigation_scheduling_method="NA" but still extract all other available information
- Return the full form of the irrigation_scheduling_method when present
- Use "NA" for values not reported, but **do not** replace valid values with "NA".
- Never write full sentences in data fields

{text}
"""

# edit units

RELEVANCE_SCORING_INSTRUCTIONS = """
You are an assistant extracting irrigation experiment data from a research paper.

Score the paper on a scale from 1-10 (where 10 is highest) on its relevance to "Predicting Crop Quality Using Irrigation Scheduling Methods".

For the relevance_score, specifically consider:
- How central irrigation scheduling methods are to the paper
- Whether the paper includes results on: irrigation_method, number_of_irrigation_events, amount_irrigated, ET_levels, daily_drainage, fruit_quality, vitamin_levels, plant_yield, avg_fruits_per_plant, avg_fruit_weight
- The comprehensiveness of the reported results
- The paper's focus on crop quality prediction using irrigation scheduling

Provide a brief justification for your score in the score_justification field.

RULES:
- For papers without irrigation focus, provide a fair relevance score based on any related content
- Use "NA" for values not reported
- Never write full sentences in data fields

{text}
"""

REFERENCE_EXTRACTION_INSTRUCTIONS = """
You are a research assistant extracting reference information from a research paper.

Extract all citations and references from the following research paper text.
Focus on identifying DOIs, titles, authors, publication year, and publication venue.
If a DOI is available, prioritize that as it's the most reliable identifier.
For each reference, include as much of the following as possible:
- Title of the paper
- Authors (as a list if possible)
- Publication year
- DOI (in format like 10.xxxx/xxxxx or as URL https://doi.org/10.xxxx/xxxxx)
- Publication venue (journal or conference)

Format each reference as a structured object.
Be accurate and comprehensive.

Paper text:
{text}
"""

# Helper Functions

In [ ]:
def setup_llm() -> Optional[ChatOpenAI]:
    if not os.environ["OPENAI_API_KEY"]:
        print("ERROR: OPENAI_API_KEY is not set. Please set your OpenAI API key.")
        return None

    try:
        # Set up the OpenAI model
        return ChatOpenAI(model="gpt-4o-mini", temperature=0)
    except Exception as e:
        print(f"Error setting up LLM: {e}")
        return None

def extract_text_from_pdf(pdf_path: str) -> str:
    print(f"\nAttempting to extract text from PDF file: {pdf_path}")
    print(f"File exists: {os.path.exists(pdf_path)}")
    print(f"File size: {os.path.getsize(pdf_path) if os.path.exists(pdf_path) else 'N/A'} bytes")

    try:
        pdf_text = ""
        # Read the PDF
        with open(pdf_path, 'rb') as file_:
            print(f"Successfully opened PDF file")
            reader = PyPDF2.PdfReader(file_)
            num_pages = len(reader.pages)
            print(f"PDF has {num_pages} pages")

            # Extract the text
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                page_text = page.extract_text()
                pdf_text += page_text
                if page_num == 0:
                    print(f"First page extract length: {len(page_text)} characters")

        final_text = pdf_text.strip().replace("\n", " ")
        print(f"Total extracted text length: {len(final_text)} characters")
        return final_text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        import traceback
        traceback.print_exc()
        return ""

def extract_text_from_file(file_path: str) -> str:
    """
    Extract text from a file (PDF or text).

    Args:
        file_path: Path to the file

    Returns:
        Extracted text
    """
    if file_path.lower().endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    else:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read().strip()
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return ""

# Main Crawler Functions

In [ ]:
def extract_features(text: str, llm: ChatOpenAI, split_into_chunks: bool = False) -> List[FeatureItem]:
    """
    Extract features and references from paper text using an LLM.

    Args:
        text: Paper text to analyze
        llm: LLM instance to use for extraction
        reference_only: If True, only extract references
        split_into_chunks: If True, split text into chunks for processing

    Returns:
        Tuple containing (features, references) as lists of Pydantic models
    """
    features = []

    try:
        structured_llm = llm.with_structured_output(FeatureLLM)

        if split_into_chunks:
            chunk_size = min(len(text) // 5, 50000)
            text_chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            for chunk in text_chunks:
                try:
                    system_message_prompt = [SystemMessage(content=FEATURE_EXTRACTION_INSTRUCTIONS.format(text=chunk))]
                    invoke_results = structured_llm.invoke(system_message_prompt)
                    features.extend(invoke_results.feature_set)
                except Exception as chunk_error:
                    print(f"Error processing chunk: {chunk_error}")
                    try:
                      if "rate" in str(chunk_error).lower():
                        time.sleep(60)
                        print("Retrying with chunk...")
                        system_message_prompt = [SystemMessage(content=FEATURE_EXTRACTION_INSTRUCTIONS.format(text=chunk))]
                        invoke_results = structured_llm.invoke(system_message_prompt)
                        features.extend(invoke_results.feature_set)
                    except Exception as e:
                      print(f"Error processing chunk: {e}")
        else:
            try:
                system_message_prompt = [SystemMessage(content=FEATURE_EXTRACTION_INSTRUCTIONS.format(text=text))]
                invoke_results = structured_llm.invoke(system_message_prompt)
                features = invoke_results.feature_set
            except Exception as feature_error:
                print(f"Error extracting features: {feature_error}")
                if "token" in str(feature_error).lower():
                    print("Text too long, retrying with chunking...")
                    return extract_features(text, llm, True)

    except Exception as e:
        print(f"Error in extraction: {e}")

        # If it's a token limit error, try splitting the text
        if not split_into_chunks and "token" in str(e).lower():
            print("Retrying with text chunking...")
            return extract_features(text, llm, True)

    return features

def process_paper(file_path: str, doi: str, llm: ChatOpenAI, content_source: str = "pdf") -> Dict[str, Any]:
    """
    Process a single paper to extract features and references.

    Args:
        file_path: Path to the paper file
        doi: DOI of the paper
        llm: LLM instance to use for extraction
        reference_only: If True, only extract references

    Returns:
        Dictionary with processing results
    """
    print(f"Processing paper: {file_path}")

    # Extract text from the paper
    text = extract_text_from_file(file_path)

    if not text or len(text) < 300:
        print(f"Insufficient text extracted from {file_path}")
        return {"features": []}

    # Extract features and references
    features = extract_features(text, llm)
    if len(features) == 0:
        return {"features": []}

    # Convert Pydantic models to dictionaries and add DOI to each feature
    feature_dicts = []
    for feature in features:
        feature_dict = feature.dict()
        feature_dict["doi_url"] = doi
        feature_dicts.append(feature_dict)

    result = {
        "features": feature_dicts,
    }

    return result

# Main Crawler

In [ ]:
reference_queue = []

full_dataframe = pd.DataFrame()
dir_path = "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files"
for filename in os.listdir(dir_path):
  if filename.endswith(".csv") and not filename.startswith("features") and not filename.startswith("all_feature"):
    file_path = os.path.join(dir_path, filename)
    try:
      df = pd.read_csv(file_path)
    except pd.errors.EmptyDataError:
      continue
    full_dataframe = pd.concat([full_dataframe, df], ignore_index=True)
    # for index, row in df.iterrows():
    #   feature = ast.literal_eval(row["features"])
    #   if len(feature) == 0:
    #     continue
    #   score = feature[0].get("relevance_score")
    #   if score < 8:
    #     continue

    #   references = ast.literal_eval(row["references"])
    #   if len(references) == 0:
    #     continue
    #   for ref in references:
    #     ref["START_PDF_PATH"] = row["file_path"]
    #   reference_queue.extend(references)
full_dataframe

,doi,file_path,features,references,processed_at,content source
0,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf
1,seed-local:e3c05b96-64d7-41e5-9bb9-04c829904fd4,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf
2,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:02:30,pdf
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf
...,...,...,...,...,...,...
160,seed-local:7afa289c-8a88-4489-98e9-24738c197508,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf
161,seed-local:9066b3a4-9f23-42bd-ad61-29000ce96183,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf
162,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:20:30,pdf
163,https://doi.org/10.1007/s13593-015-0338-6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'Regulated deficit irrigation for c...,2025-09-13 00:21:22,pdf


In [ ]:
import ast
# create a relevance_score column
def get_revelance_score(row):
  feature = ast.literal_eval(row["features"])
  if len(feature) == 0:
    return 0
  score = feature[0].get("relevance_score")
  return score

full_dataframe["relevance_score"] = full_dataframe.apply(get_revelance_score, axis=1)
full_dataframe

,doi,file_path,features,references,processed_at,content source,relevance_score
0,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9
1,seed-local:e3c05b96-64d7-41e5-9bb9-04c829904fd4,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9
2,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:02:30,pdf,9
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9
...,...,...,...,...,...,...,...
160,seed-local:7afa289c-8a88-4489-98e9-24738c197508,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9
161,seed-local:9066b3a4-9f23-42bd-ad61-29000ce96183,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf,8
162,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:20:30,pdf,8
163,https://doi.org/10.1007/s13593-015-0338-6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'Regulated deficit irrigation for c...,2025-09-13 00:21:22,pdf,8


In [ ]:
full_dataframe = full_dataframe.loc[full_dataframe["relevance_score"] >= 8]
full_dataframe

,doi,file_path,features,references,processed_at,content source,relevance_score
0,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9
1,seed-local:e3c05b96-64d7-41e5-9bb9-04c829904fd4,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9
2,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:02:30,pdf,9
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9
...,...,...,...,...,...,...,...
160,seed-local:7afa289c-8a88-4489-98e9-24738c197508,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9
161,seed-local:9066b3a4-9f23-42bd-ad61-29000ce96183,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf,8
162,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:20:30,pdf,8
163,https://doi.org/10.1007/s13593-015-0338-6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'Regulated deficit irrigation for c...,2025-09-13 00:21:22,pdf,8


In [ ]:
# check for duplicate file_paths
print(f"{len(full_dataframe['file_path'])} VS {len(set(full_dataframe['file_path']))}")
print(f"{len(full_dataframe['doi'])} VS {len(set(full_dataframe['doi']))}")

df = full_dataframe.copy()
mask = df.duplicated(subset=["references"]) & df["references"].apply(lambda x: isinstance(x, list) and len(x) > 0)
print(f"{len(full_dataframe['references'])} VS {len(df[~mask])}")

164 VS 164
164 VS 159
164 VS 164


In [ ]:
full_dataframe = full_dataframe.drop_duplicates(subset=["doi"])
full_dataframe

,doi,file_path,relevance_dict,references,processed_at,content source,relevance_score
0,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9
1,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9
5,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of water stress at differen...,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...
158,https://doi.org/10.1371/journal.pone.0154799,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Surface Drainage and Mulching Drip...,2025-09-13 00:15:09,web,9
159,https://doi.org/10.1371/journal.pone.0142204,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'The Effects of Saline Water Drip I...,2025-09-13 00:16:15,web,9
160,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9
161,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf,8


In [ ]:
for index, row in full_dataframe.iterrows():
  if not row["doi"].startswith("NA") and not row["doi"].startswith("seed"):
    continue

  file_path = row["file_path"]
  print("Processing paper: {}".format(file_path))

  if file_path == "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/Seed Papers/Influence_of_Irrigation_Scheduling_on_Pr.pdf":
    full_dataframe.at[index, "doi"] = "http://dx.doi.org/10.5281/zenodo.7790864"
    print("changed")
    continue

  class DOILLM(BaseModel):
    doi_url: str = Field(description="DOI URL if available, otherwise 'NA'")

  DOI_EXTRACTION_INSTRUCTIONS = """
You are a research assistant tasked with identifying the doi of a given paper.

Return a DOI URL in this format https://doi.org/10.xxxx/xxxxx

Paper text:
{text}
"""

  llm = setup_llm()
  structured_llm = llm.with_structured_output(DOILLM)
  system_message_prompt = [SystemMessage(content=DOI_EXTRACTION_INSTRUCTIONS.format(text=extract_text_from_file(file_path)))]
  invoke_results = structured_llm.invoke(system_message_prompt)

  print(invoke_results.doi_url)
  full_dataframe.at[index, "doi"] = invoke_results.doi_url


In [ ]:
full_dataframe.rename(columns={'features' : 'relevance_dict'}, inplace= True)

/tmp/ipython-input-1578043444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dataframe.rename(columns={'features' : 'relevance_dict'}, inplace= True)


In [ ]:
full_dataframe

,doi,file_path,relevance_dict,references,processed_at,content source,relevance_score,relevance_dict
0,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,{'features': [{'irrigation_scheduling_method':...
1,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,{'features': [{'irrigation_scheduling_method':...
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8,{'features': [{'irrigation_scheduling_method':...
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9,{'features': [{'irrigation_scheduling_method':...
5,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of water stress at differen...,2025-09-12 04:06:41,pdf,9,{'features': [{'irrigation_scheduling_method':...
...,...,...,...,...,...,...,...,...
158,https://doi.org/10.1371/journal.pone.0154799,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Surface Drainage and Mulching Drip...,2025-09-13 00:15:09,web,9,{'features': [{'irrigation_scheduling_method':...
159,https://doi.org/10.1371/journal.pone.0142204,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'The Effects of Saline Water Drip I...,2025-09-13 00:16:15,web,9,{'features': [{'irrigation_scheduling_method':...
160,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9,{'features': [{'irrigation_scheduling_method':...
161,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf,8,{'features': [{'irrigation_scheduling_method':...


In [ ]:
full_dataframe.loc[full_dataframe["content source"] == "pdf"]

,doi,file_path,relevance_dict,references,processed_at,content source,relevance_score
0,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9
1,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9
3,https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8
4,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9
5,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Effect of water stress at differen...,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...
150,https://doi.org/10.13031/trans.14110,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'Irrigation Scheduling for Agricult...,2025-09-12 23:57:48,pdf,8
151,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 8, 'score_justification':...",[{'title': 'Fruit yield and water use efficien...,2025-09-12 23:58:14,pdf,8
153,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9
160,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"[{'relevance_score': 9, 'score_justification':...",[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9


In [ ]:
llm = setup_llm()

import warnings

warnings.filterwarnings(
    "ignore",
    message="Received a Pydantic BaseModel V1 schema",
    category=UserWarning,
)

def main(row):
    features = process_paper(row["file_path"], row["doi"], llm)
    print(features)
    return features

full_dataframe["features"] = full_dataframe.apply(main, axis=1)

pdf_basename = "features_compiled"
csv_filename = f"{pdf_basename}_irrigation_dataset (3).csv"

SAVE_LOCATION = os.path.join("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis", csv_filename)

try:
  full_dataframe.to_csv(SAVE_LOCATION, index = False)
  print(f"Dataset saved to: {SAVE_LOCATION}")
except Exception as e:
  try:
    with open(csv_filename, "wb") as f:
      pickle.dump(full_dataframe, f)
    print(f"Dataset saved to: {SAVE_LOCATION}")
  except Exception as e:
    print(f"Error saving dataset: {e}")

Processing paper: /content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/Seed Papers/Effects of irrigation scheduling on the yield and irrigation water productivity of cucumber in coconut coir culture (1).pdf

Attempting to extract text from PDF file: /content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/Seed Papers/Effects of irrigation scheduling on the yield and irrigation water productivity of cucumber in coconut coir culture (1).pdf
File exists: True
File size: 2136079 bytes
Successfully opened PDF file
PDF has 13 pages
First page extract length: 4333 characters
Total extracted text length: 58044 characters
{'features': [{'irrigation_scheduling_method': 'Soil moisture sensor-based method (T-VWC)', 'sensor_type': 'Soil moisture sensor', 'sensor_threshold': 'VWC ≤ VWC max', 'number_of_sensors': '1', 'evapotranspiration_level': 'ETd = 4.63 mm (beginning), 3.79 mm (middle), 3.63 mm (end)', 'daily_drainage': '0.34-0.43 L plant−1', 'amount_irrigated': '1.70-2.00 L plant−1', 'number_of_irrigation_eve

KeyboardInterrupt: 

In [ ]:
all_samples = []
for index, row in full_dataframe.iterrows():
  features = row["features"]
  if len(features) == 0 or len(features.get("features")) == 0:
    continue
  features = features.get("features")
  for sample in features:
    sample["file_path"] = row["file_path"]
    sample["doi"] = row["doi"]
    sample["references"] = ast.literal_eval(row["references"])
    sample["processed_at"] = row["processed_at"]
    sample["content source"] = row["content source"]
    sample["relevance_score"] = row["relevance_score"]
    sample["relevance_dict"] = row["relevance_dict"]
    all_samples.append(sample)

pd.DataFrame(all_samples)

,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,location,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,"ETd = 4.63 mm (beginning), 3.79 mm (middle), 3...",0.34-0.43 L plant−1,"1.70-2.00 L plant−1 (beginning), 2.00 L plant−...",12,07:00–18:00,Xiaotangshan National Precision Agriculture Ex...,...,NA,17.37 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,"ETd = 4.66 mm (beginning), 3.03 mm (middle), 1...","0.40 L plant−1 (beginning), 0.34 L plant−1 (mi...","2.03 L plant−1 (beginning), 1.40 L plant−1 (mi...","14 (beginning), 9 (middle), 5 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,20.52 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,Crop evapotranspiration estimated method using...,NA,ETc h ≥ 0.4 mm,NA,"ETd = 2.94 mm (beginning), 2.17 mm (middle), 2...","0.26 L plant−1 (beginning), 0.36 L plant−1 (mi...","1.29 L plant−1 (beginning), 1.12 L plant−1 (mi...","9 (beginning), 9 (middle), 10 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,25.07 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,Soil water balance method,"Neutron probe, gravimetric method",50% depletion of total available soil moisture,NA,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NA,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),"Melkassa Agricultural Research Center, Awash B...",...,NA,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
4,NA,Neutron probe,NA,NA,2000 mm year-1,NA,"753 mm in 2017, 700 mm in 2018",NA,Irrigation started one month after sowing,"Agricultural Experiment Station, Damascus, Syria",...,NA,"IWUEhc: 1.97 kg m-3 (S-75), 1.65 kg m-3 (S-150...",https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2006.03.019,[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8,"[{'relevance_score': 8, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Deficit Irrigation (DI),NA,NA,NA,NA,NA,NA,NA,NA,Global,...,NA,8.1% to 30.1% gains in water productivity (WP)...,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-021-01433-w,[{'title': 'A global meta-analysis of yield an...,2025-09-13 00:19:30,pdf,8,"[{'relevance_score': 8, 'score_justification':..."
601,Deficit Irrigation (DI),NA,NA,NA,70% to 90% ETc,NA,NA,NA,NA,NA,...,NA,Improved WUE with small water deficits,https://doi.org/10.21273/hortsci14271-19,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21273/hortsci14271-19,[{'title': 'Potential of Integr

In [ ]:
all_samples = pd.DataFrame(all_samples)
all_samples.loc[all_samples["content source"] == "pdf"]

,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,location,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,"ETd = 4.63 mm (beginning), 3.79 mm (middle), 3...",0.34-0.43 L plant−1,"1.70-2.00 L plant−1 (beginning), 2.00 L plant−...",12,07:00–18:00,Xiaotangshan National Precision Agriculture Ex...,...,NA,17.37 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,"ETd = 4.66 mm (beginning), 3.03 mm (middle), 1...","0.40 L plant−1 (beginning), 0.34 L plant−1 (mi...","2.03 L plant−1 (beginning), 1.40 L plant−1 (mi...","14 (beginning), 9 (middle), 5 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,20.52 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,Crop evapotranspiration estimated method using...,NA,ETc h ≥ 0.4 mm,NA,"ETd = 2.94 mm (beginning), 2.17 mm (middle), 2...","0.26 L plant−1 (beginning), 0.36 L plant−1 (mi...","1.29 L plant−1 (beginning), 1.12 L plant−1 (mi...","9 (beginning), 9 (middle), 10 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,25.07 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,Soil water balance method,"Neutron probe, gravimetric method",50% depletion of total available soil moisture,NA,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NA,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),"Melkassa Agricultural Research Center, Awash B...",...,NA,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
4,NA,Neutron probe,NA,NA,2000 mm year-1,NA,"753 mm in 2017, 700 mm in 2018",NA,Irrigation started one month after sowing,"Agricultural Experiment Station, Damascus, Syria",...,NA,"IWUEhc: 1.97 kg m-3 (S-75), 1.65 kg m-3 (S-150...",https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2006.03.019,[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8,"[{'relevance_score': 8, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,Deficit Irrigation (Soil Texture: Loam),NA,NA,NA,NA,NA,NA,NA,NA,"Xinjiang, China",...,NA,13.51%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
580,Deficit Irrigation (Soil Texture: Sandy),NA,NA,NA,NA,NA,NA,NA,NA,"Xinjiang, China",...,NA,8.83%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pd

In [ ]:
valid_samples = all_samples.loc[all_samples["irrigation_scheduling_method"] != "NA"]
valid_samples

,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,location,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,"ETd = 4.63 mm (beginning), 3.79 mm (middle), 3...",0.34-0.43 L plant−1,"1.70-2.00 L plant−1 (beginning), 2.00 L plant−...",12,07:00–18:00,Xiaotangshan National Precision Agriculture Ex...,...,NA,17.37 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,"ETd = 4.66 mm (beginning), 3.03 mm (middle), 1...","0.40 L plant−1 (beginning), 0.34 L plant−1 (mi...","2.03 L plant−1 (beginning), 1.40 L plant−1 (mi...","14 (beginning), 9 (middle), 5 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,20.52 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,Crop evapotranspiration estimated method using...,NA,ETc h ≥ 0.4 mm,NA,"ETd = 2.94 mm (beginning), 2.17 mm (middle), 2...","0.26 L plant−1 (beginning), 0.36 L plant−1 (mi...","1.29 L plant−1 (beginning), 1.12 L plant−1 (mi...","9 (beginning), 9 (middle), 10 (end)",NA,Xiaotangshan National Precision Agriculture Ex...,...,NA,25.07 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,Soil water balance method,"Neutron probe, gravimetric method",50% depletion of total available soil moisture,NA,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NA,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),"Melkassa Agricultural Research Center, Awash B...",...,NA,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
5,Alternate Furrow Irrigation (AFI),NA,NA,NA,450.0 mm,NA,2100 m3/hm2,7,1997-2000,"Inland River Basin, Northwest China",...,NA,NA,https://doi.org/10.1016/j.agwat.2009.08.011,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2009.08.011,[{'title': 'An improved water-use efficiency o...,2025-09-12 04:05:45,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,Film-Mulched Drip Irrigation,NA,NA,NA,"60%, 80%, 100%, 120% (W 0.6, W 0.8, W 1.0, W 1.2)",NA,"219.20 mm (W 0.6), 281.26 mm (W 0.8), 343.33 m...",NA,15 June to 27 August,"Korla, Xinjiang, China",...,NA,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,[{'title': 'Evaluating of Four Irrigation Dept...,2025-09-13 00:18:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
600,Deficit Irrigation (DI),NA,NA,NA,NA,NA,NA,NA,NA,Global,...,NA,8.1% to 30.1% gains in water productivity (WP)...,https://doi.org/10.1038/s41598-021-01433-w,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-0

In [ ]:
pd.DataFrame(all_samples).to_csv("all_feature_samples (2).csv")

In [ ]:
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/all_feature_samples.csv")
df["irrigation_scheduling_method"] = df["irrigation_scheduling_method"].apply(lambda x: np.nan if x == "NA" else x)
df.dropna(subset=["irrigation_scheduling_method"], inplace=True)
df

In [ ]:
len(set(df["plant_type"])) - 1